In [4]:
import wandb
wandb.login(key=input())

wandb: Currently logged in as: gabi-kadlecova. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/gabi/.netrc


True

In [57]:
project = 'zc_nb201'

api = wandb.Api(timeout=19)
runs = api.runs('gabi-kadlecova' + "/" + project)

In [58]:
for run in runs:
    # skip xgb for now
    if run.config['model'] != 'rf':
        continue
    
    print(run.summary['corr_mean'])
    print(run.config)
    break

0.9572264642211716
{'cfg': '../zc_combine/configs/nb201_full.json', 'meta': '../data/meta.json', 'seed': 42, 'model': 'rf', 'proxy': 'fisher', 'dataset': 'cifar10', 'n_evals': 10, 'features': None, 'benchmark': 'nb201', 'data_seed': 46, 'out_prefix': 'train-380', 'train_size': 1000, 'version_key': 'train_version', 'keep_uniques': True, 'use_features': True, 'use_all_proxies': False, 'use_flops_params': True, 'zero_unreachables': True}


In [59]:
import pandas as pd

def create_entry(run):
    # TODO uncertainty
    row = {}
    cfg_args = ['cfg', 'data_seed', 'proxy', 'train_size', 'use_all_proxies', 'use_features']
    for ca in cfg_args:
        row[ca] = run.config[ca]

    prox, cfg = row['proxy'], row['cfg']
    row['proxy'] = 'NONE' if prox is None or not len(prox) else prox
    row['cfg'] = 'NONE' if cfg is None or not len(cfg) else cfg
    row['corr'] = run.summary['corr_mean']
    row['tau'] = run.summary['tau_mean']
    return row

df = [create_entry(r) for r in runs]
df = pd.DataFrame(df)
df

,cfg,data_seed,proxy,train_size,use_all_proxies,use_features,corr,tau
0,../zc_combine/configs/nb201_full.json,46,fisher,1000,False,True,0.957226,0.829190
1,../zc_combine/configs/nb201_first.json,46,jacov,1000,False,True,0.958952,0.828732
2,../zc_combine/configs/nb201_full.json,46,plain,100,False,True,0.899698,0.735508
3,../zc_combine/configs/nb201_first.json,46,zen,100,False,True,0.889051,0.715066
4,../zc_combine/configs/nb201_first.json,46,epe_nas,100,False,True,0.910811,0.745794
...,...,...,...,...,...,...,...,...
415,NONE,42,NONE,1000,True,False,0.940985,0.792016
416,NONE,42,NONE,100,True,False,0.901894,0.729372
417,NONE,42,NONE,100,True,False,0.914283,0.746074
418,NONE,42,NONE,20,True,False,0.873683,0.685865


In [61]:
what_cfg = 'full'
group_cols = ['train_size', 'use_all_proxies', 'use_features', 'proxy']

small_df = df[(df['cfg'].str.contains('NONE')) | (df['cfg'].str.contains(what_cfg))].drop(columns='cfg')
small_df.groupby(group_cols).mean()

data_seed      corr  \
train_size use_all_proxies use_features proxy                            
20         False           True         NONE            44.0  0.791740   
                                        epe_nas         44.0  0.840718   
                                        fisher          44.0  0.793019   
                                        grad_norm       44.0  0.792815   
                                        grasp           44.0  0.800115   
                                        jacov           44.0  0.844494   
                                        l2_norm         44.0  0.799439   
                                        nwot            44.0  0.804457   
                                        plain           44.0  0.792867   
                                        snip            44.0  0.793850   
                                        synflow         44.0  0.798391   
                                        zen             44.0  0.790781   
           True            False        NONE            44.0  0.866033   
                           True         NONE            44.0  0.861470   
100        False           True         NONE            44.0  0.892574   
                                        epe_nas         44.0  0.915581   
                                        fisher          44.0  0.896590   
                                        grad_norm       44.0  0.904397   
                                        grasp           44.0  0.900985   
                                        jacov           44.0  0.923602   
                                        l2_norm         44.0  0.886401   
                                        nwot            44.0  0.902417   
                                        plain           44.0  0.893116   
                                        snip            44.0  0.907887   
                                        synflow         44.0  0.890607   
                                        zen             44.0  0.896074   
           True            False        NONE            44.0  0.905981   
                           True         NONE            44.0  0.918346   
1000       False           True         NONE            44.0  0.954134   
                                        epe_nas         44.0  0.957743   
                                        fisher          44.0  0.956497   
                                        grad_norm       44.0  0.959654   
                                        grasp           44.0  0.958201   
                                        jacov           44.0  0.962535   
                                        l2_norm         44.0  0.952061   
                                        nwot            44.0  0.956534   
                                        plain           44.0  0.952528   
                                        snip            44.0  0.961882   
                                        synflow         44.0  0.956127   
                                        zen             44.0  0.958489   
           True            False        NONE            44.0  0.944283   
                           True         NONE            44.0  0.962655   

                                                        tau  
train_size use_all_proxies use_features proxy                
20         False           True         NONE       0.601163  
                                        epe_nas    0.650994  
                                        fisher     0.602359  
                                        grad_norm  0.602378  
                                        grasp      0.610184  
                                        jacov      0.654127  
                                        l2_norm    0.608268  
                                        nwot       0.613428  
                                        plain      0.602274  
                                        snip       0.603436  
                                        synflow   